In [1]:
# calculate potential savings in USD from moving your existing RDS fleet to Aurora Serverless V2
#
# BUGS
# - hard-wired Aurora Serverless V2 ACU pricing
# - assumes all On-Demand, does not factor in Reserved Instances
# - assumes 1 ACU = 0.25 vCPU which (probably) is fine for now but may change
# - doesn't work well for burstable instances (assumes that they are equivalent to regular instances)
# - not tested on account with large number of DB instances (does describe_db_instances paginate?)
# - requires that your AWS CLI is properly set up and with proper AWS access/secret keys and region
# - only queries RDS fleet in current region (defined in AWS CLI configuration)
#
# NEW FEATURE
# - calculates IOPS cost when moving from EBS storage to Aurora storage
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED
# TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHOR OR COPYRIGHT HOLDER BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF
# CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS
# IN THE SOFTWARE.
#
# receiver of blame: orly.andico@gmail.com

import boto3
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import json
import math
import os

client = boto3.client('rds')
cw = boto3.client('cloudwatch')
sess = boto3.session.Session()
region = sess.region_name


pd.set_option("display.max_columns", None)
print(region)

ap-southeast-1


In [2]:
response = client.describe_db_instances()

In [3]:
df = pd.DataFrame()
df = pd.concat([df, pd.DataFrame(response['DBInstances']) ], ignore_index=True)
df

,DBInstanceIdentifier,DBInstanceClass,Engine,DBInstanceStatus,MasterUsername,Endpoint,AllocatedStorage,InstanceCreateTime,PreferredBackupWindow,BackupRetentionPeriod,DBSecurityGroups,VpcSecurityGroups,DBParameterGroups,AvailabilityZone,DBSubnetGroup,PreferredMaintenanceWindow,PendingModifiedValues,MultiAZ,EngineVersion,AutoMinorVersionUpgrade,ReadReplicaDBInstanceIdentifiers,LicenseModel,OptionGroupMemberships,PubliclyAccessible,StorageType,DbInstancePort,DBClusterIdentifier,StorageEncrypted,KmsKeyId,DbiResourceId,CACertificateIdentifier,DomainMemberships,CopyTagsToSnapshot,MonitoringInterval,PromotionTier,DBInstanceArn,IAMDatabaseAuthenticationEnabled,PerformanceInsightsEnabled,DeletionProtection,AssociatedRoles,TagList,CustomerOwnedIpEnabled,BackupTarget,NetworkType,StorageThroughput,CertificateDetails,LatestRestorableTime,Iops,ActivityStreamStatus,SecondaryAvailabilityZone,EnhancedMonitoringResourceArn,MonitoringRoleArn,PerformanceInsightsKMSKeyId,PerformanceInsightsRetentionPeriod
0,a1,db.r6g.xlarge,aurora-mysql,available,orly,{'Address': 'a1.ccwiwddgdpvf.ap-southeast-1.rd...,1,2023-03-18 04:45:36.264000+00:00,20:08-20:38,1,[],[{'VpcSecurityGroupId': 'sg-0ccfa165eadcbfc01'...,[{'DBParameterGroupName': 'default.aurora-mysq...,ap-southeast-1a,"{'DBSubnetGroupName': 'privatesubnetgroup', 'D...",fri:18:25-fri:18:55,{},False,8.0.mysql_aurora.3.03.0,True,[],general-public-license,[{'OptionGroupName': 'default:aurora-mysql-8-0...,False,aurora,0,a1-cluster,True,arn:aws:kms:ap-southeast-1:773521480638:key/bc...,db-Z2LTE3RWYATZD5WHWYBVWCFG3I,rds-ca-2019,[],False,0,1.0,arn:aws:rds:ap-southeast-1:773521480638:db:a1,False,False,False,[],[],False,region,IPV4,0,"{'CAIdentifier': 'rds-ca-2019', 'ValidTill': 2...",NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a2maz,db.r6g.xlarge,aurora-mysql,available,orly,{'Address': 'a2maz.ccwiwddgdpvf.ap-southeast-1...,1,2023-03-20 03:32:14.134000+00:00,18:47-19:17,1,[],[{'VpcSecurityGroupId': 'sg-0ccfa165eadcbfc01'...,[{'DBParameterGroupName': 'default.aurora-mysq...,ap-southeast-1a,"{'DBSubnetGroupName': 'privatesubnetgroup', 'D...",fri:18:54-fri:19:24,{},False,8.0.mysql_aurora.3.03.0,True,[],general-public-license,[{'OptionGroupName': 'default:aurora-mysql-8-0...,False,aurora,0,a2maz-cluster,True,arn:aws:kms:ap-southeast-1:773521480638:key/bc...,db-QH6RQP2II37HPLF56IFRS7G3J4,rds-ca-2019,[],False,0,1.0,arn:aws:rds:ap-southeast-1:773521480638:db:a2maz,False,False,False,[],[],False,region,IPV4,0,"{'CAIdentifier': 'rds-ca-2019', 'ValidTill': 2...",NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a2maz-ap-southeast-1b,db.r6g.xlarge,aurora-mysql,available,orly,{'Address': 'a2maz-ap-southeast-1b.ccwiwddgdpv...,1,2023-03-20 03:37:38.872000+00:00,18:47-19:17,1,[],[{'VpcSecurityGroupId': 'sg-0ccfa165eadcbfc01'...,[{'DBParameterGroupName': 'default.aurora-mysq...,ap-southeast-1b,"{'DBSubnetGroupName': 'privatesubnetgroup', 'D...",sun:16:28-sun:16:58,{},False,8.0.mysql_aurora.3.03.0,True,[],general-public-license,[{'OptionGroupName': 'default:aurora-mysql-8-0...,False,aurora,0,a2maz-cluster,True,arn:aws:kms:ap-southeast-1:773521480638:key/bc...,db-2ZKYXJE662VOZ7SEDPIBJFLKYA,rds-ca-2019,[],False,0,1.0,arn:aws:rds:ap-southeast-1:773521480638:db:a2m...,False,False,False,[],[],False,region,IPV4,0,"{'CAIdentifier': 'rds-ca-2019', 'ValidTill': 2...",NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,g1,db.r6g.xlarge,mysql,available,orly,{'Address': 'g1.ccwiwddgdpvf.ap-southeast-1.rd...,500,2023-03-18 03:39:24.068000+00:00,17:36-18:06,7,[],[{'VpcSecurityGroupId': 'sg-0ccfa165eadcbfc01'...,"[{'DBParameterGroupName': 'default.mysql8.0', ...",ap-southeast-1b,"{'DBSubnetGroupName': 'privatesubnetgroup', 'D...",thu:18:42-thu:19:12,{},False,8.0.28,True,[],general-public-license,"[{'OptionGroupName': 'default:mysql-8-0', 'Sta...",False,gp3,0,NaN,True,arn:aws:kms:ap-southeast-1:773521480638:key/bc...,db-NUGJOCO4YSI4JVPYGC6TJTWX3Q,rds-ca-2019,[],True,0,NaN,arn:aws:rds:ap-southeast-1:773521480638:db:g1,False,False,False,[],[],False,region,IPV4,500,"{'CAIdentifier': 'rds-c

In [4]:
df2 = df.filter(['DBInstanceIdentifier','DBInstanceClass','Engine', 'DBInstanceStatus', 'AllocatedStorage', 'SecondaryAvailabilityZone'], axis=1)
df2 = df2.astype({"AllocatedStorage": int, "SecondaryAvailabilityZone": str})

# we need to remove DBInstanceClass = "db.serverless" which corresponds to serverless v2
# serverless v1 does not show up in describe_db_instances
df2 = df2[ df2['DBInstanceClass'] != 'db.serverless'].reset_index(drop=True)

# we also need to remove any databases which aren't supported (i.e. not MySQL or PostgreSQL)
df2 = df2[ df2['Engine'].isin(['aurora-mysql', 'mysql', 'aurora-postgresql', 'postgresql'])]

df2

,DBInstanceIdentifier,DBInstanceClass,Engine,DBInstanceStatus,AllocatedStorage,SecondaryAvailabilityZone
0,a1,db.r6g.xlarge,aurora-mysql,available,1,nan
1,a2maz,db.r6g.xlarge,aurora-mysql,available,1,nan
2,a2maz-ap-southeast-1b,db.r6g.xlarge,aurora-mysql,available,1,nan
3,g1,db.r6g.xlarge,mysql,available,500,nan
4,g2,db.r6g.xlarge,mysql,available,500,ap-southeast-1c
5,i1,db.r5.xlarge,mysql,available,500,nan


In [5]:
# extract pricing data
# this only really works for MySQL and PostgreSQL because we hard-wire the no license required
def get_rds_instance_hourly_price(region_name, instance_type, database_engine, deployment_option):

    filters = [
        {'Type': 'TERM_MATCH', 'Field': 'instanceType', 'Value': instance_type},
        {'Type': 'TERM_MATCH', 'Field': 'databaseEngine', 'Value': database_engine},
        {'Type': 'TERM_MATCH', 'Field': 'licenseModel', 'Value': 'No License required'},
        {'Type': 'TERM_MATCH', 'Field': 'deploymentOption', 'Value': deployment_option},        
        {'Type': 'TERM_MATCH', 'Field': 'regionCode', 'Value': region_name}
    ]
    
#    print ("DEBUG: ", filters)

    pricing_client = boto3.client('pricing', region_name='us-east-1')    
    response = pricing_client.get_products(ServiceCode='AmazonRDS', Filters=filters, MaxResults=1)

    j = json.loads(response['PriceList'][0])
    od = j['terms']['OnDemand']
    id1 = list(od)[0]
    id2 = list(od[id1]['priceDimensions'])[0]

    price_od = od[id1]['priceDimensions'][id2]['pricePerUnit']['USD']

    r = {
        'vcpu': j['product']['attributes']['vcpu'],
        'memory': j['product']['attributes']['memory'],
        'pricePerUnit': price_od,
        'instanceType': j['product']['attributes']['instanceType'],
        'databaseEngine': j['product']['attributes']['databaseEngine'],
        'deploymentOption': j['product']['attributes']['deploymentOption']
    }
    return (r)

In [6]:
# fetch the pricing for every row in the RDS instances

nrows = len(df2.index)
idx = 0

# iterate over all rows in dataframe
while (idx < nrows):
    db = df2.iloc[idx]['Engine']
    az = df2.iloc[idx]['SecondaryAvailabilityZone']
    if (len(az) > 4):
        deploymentOption = 'Multi-AZ'
    else:
        deploymentOption = 'Single-AZ'
    
    ic = df2.iloc[idx]['DBInstanceClass']
    
    if (db == 'mysql'):
        databaseEngine = 'MySQL'
    elif (db == 'aurora-mysql'):
        databaseEngine = 'Aurora MySQL'
    elif (db == 'postgresql'):
        databaseEngine = 'PostgreSQL'
    elif (db == 'aurora-postgresql'):
        databaseEngine = 'Aurora PostgreSQL'
    else:
        databaseEngine = 'MySQL'
    
    
    r = get_rds_instance_hourly_price(region, ic, databaseEngine, deploymentOption)
    
    # sanity check that we got the correct match
    if (ic == r['instanceType'] and
        databaseEngine == r['databaseEngine'] and
       deploymentOption == r['deploymentOption']):
        
        r['memory'] = r['memory'].replace(" GiB", "") 
        
        r['memory'] = float(r['memory'])
        r['pricePerUnit'] = float(r['pricePerUnit'])
        r['vcpu'] = float(r['vcpu'])

        # NOTE: multi-AZ deployments have the 2 instances factored into the cost!
        # no need to multiply by 2
        if (deploymentOption == 'Multi-AZ'):
            num = 1
        else:
            num = 1
            
        print(r, "\n")
        df2.loc[idx, ['pricePerUnit', 'deploymentOption', 'vcpu', 'memory', 'pricePerMonth']] = [r['pricePerUnit'], r['deploymentOption'], r['vcpu'], r['memory'], r['pricePerUnit'] * 730 * num ]
                                                                                
    idx = idx + 1
                                                            


{'vcpu': 4.0, 'memory': 32.0, 'pricePerUnit': 0.627, 'instanceType': 'db.r6g.xlarge', 'databaseEngine': 'Aurora MySQL', 'deploymentOption': 'Single-AZ'} 

{'vcpu': 4.0, 'memory': 32.0, 'pricePerUnit': 0.627, 'instanceType': 'db.r6g.xlarge', 'databaseEngine': 'Aurora MySQL', 'deploymentOption': 'Single-AZ'} 

{'vcpu': 4.0, 'memory': 32.0, 'pricePerUnit': 0.627, 'instanceType': 'db.r6g.xlarge', 'databaseEngine': 'Aurora MySQL', 'deploymentOption': 'Single-AZ'} 

{'vcpu': 4.0, 'memory': 32.0, 'pricePerUnit': 0.51, 'instanceType': 'db.r6g.xlarge', 'databaseEngine': 'MySQL', 'deploymentOption': 'Single-AZ'} 

{'vcpu': 4.0, 'memory': 32.0, 'pricePerUnit': 1.02, 'instanceType': 'db.r6g.xlarge', 'databaseEngine': 'MySQL', 'deploymentOption': 'Multi-AZ'} 

{'vcpu': 4.0, 'memory': 32.0, 'pricePerUnit': 0.57, 'instanceType': 'db.r5.xlarge', 'databaseEngine': 'MySQL', 'deploymentOption': 'Single-AZ'} 



In [7]:
df2

,DBInstanceIdentifier,DBInstanceClass,Engine,DBInstanceStatus,AllocatedStorage,SecondaryAvailabilityZone,pricePerUnit,deploymentOption,vcpu,memory,pricePerMonth
0,a1,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71
1,a2maz,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71
2,a2maz-ap-southeast-1b,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71
3,g1,db.r6g.xlarge,mysql,available,500,nan,0.510,Single-AZ,4.0,32.0,372.30
4,g2,db.r6g.xlarge,mysql,available,500,ap-southeast-1c,1.020,Multi-AZ,4.0,32.0,744.60
5,i1,db.r5.xlarge,mysql,available,500,nan,0.570,Single-AZ,4.0,32.0,416.10


In [9]:
# we can only fetch 1440 data points from Cloudwatch, so over a 2-week period (20160 minutes)
# our sampling interval is 14 minutes; also note we are fetching the *MAXIMUM* over each sampling period
# however, let's use a less aggressive 1-hour sampling interval

# note ReadIOPS/WriteIOPS are for RDS EBS engines (and Aurora PostgreSQL)
# VolumeReadIOPS/VolumeWriteIOPS are for Aurora MySQL and PostgreSQL *but only on cluster level*

# we don't really need IOPS for Aurora because moving from Aurora Provisioned to Aurora Serverless won't change the IOPS cost
# but moving from RDS EBS engines to Aurora storage will

dfutil = pd.DataFrame()
dfiops_read = pd.DataFrame()
dfiops_write = pd.DataFrame()

for dbid in df2['DBInstanceIdentifier']:
    
    # this filter returns a series, but the series only contains one item
    engine = df2[ (df2['DBInstanceIdentifier'] == dbid) ].Engine.item()

    # CPUUtilization
    stats = cw.get_metric_statistics(
        Namespace='AWS/RDS',
        Dimensions=[
            {
                'Name': 'DBInstanceIdentifier',
                'Value': dbid
            }
        ],
        MetricName='CPUUtilization',
        StartTime=datetime.now() - timedelta(days=14),
        EndTime=datetime.now(),
        Period=3600,
        Statistics=[ 'Maximum' ])
    df3 = pd.DataFrame(stats['Datapoints'])
    df3['DBInstanceIdentifier'] = dbid

    dfutil = pd.concat([dfutil, df3], ignore_index=True)

    # skip IOPS for aurora engines
    if ( (engine != 'aurora-mysql') and (engine != 'aurora-postgresql') ):
        metricname="ReadIOPS"

        # ReadIOPS
        stats = cw.get_metric_statistics(
            Namespace='AWS/RDS',
            Dimensions=[
                {
                    'Name': 'DBInstanceIdentifier',
                    'Value': dbid
                }
            ],
            MetricName=metricname,
            StartTime=datetime.now() - timedelta(days=14),
            EndTime=datetime.now(),
            Period=3600,
            Statistics=[ 'Maximum' ])
        df3 = pd.DataFrame(stats['Datapoints'])
        df3['DBInstanceIdentifier'] = dbid

        dfiops_read = pd.concat([dfiops_read, df3], ignore_index=True)

    if ((engine != 'aurora-mysql') and (engine != 'aurora-postgresql')):
        metricname="WriteIOPS"

        # WriteIOPS
        stats = cw.get_metric_statistics(
            Namespace='AWS/RDS',
            Dimensions=[
                {
                    'Name': 'DBInstanceIdentifier',
                    'Value': dbid
                }
            ],
            MetricName=metricname,
            StartTime=datetime.now() - timedelta(days=14),
            EndTime=datetime.now(),
            Period=3600,
            Statistics=[ 'Maximum' ])
        df3 = pd.DataFrame(stats['Datapoints'])
        df3['DBInstanceIdentifier'] = dbid

        dfiops_write = pd.concat([dfiops_write, df3], ignore_index=True)

In [15]:
# for each DBInstanceIdentifier, get the average and maximum CPU utilization
df_agg = dfutil.groupby("DBInstanceIdentifier").Maximum.agg(["mean", "std", "max", "count"]).reset_index()

df_agg.columns = [ "DBInstanceIdentifier", "meanCPU", "stdCPU", "maxCPU", "countCPU" ]
df_agg

,DBInstanceIdentifier,meanCPU,stdCPU,maxCPU,countCPU
0,a1,5.590856,13.841096,99.761667,222
1,a2maz,5.087267,9.460886,99.770000,175
2,a2maz-ap-southeast-1b,3.986857,1.487108,23.253333,175
3,g1,3.600990,13.481081,99.616667,224
4,g2,2.109888,2.617055,27.020000,178
5,i1,5.323446,18.077301,99.228333,267


In [16]:
# for each DBInstanceIdentifier, get the average and maximum Read IOPS
df_agg_r = dfiops_read.groupby("DBInstanceIdentifier").Maximum.agg(["mean", "std", "max", "count"]).reset_index()

df_agg_r.columns = [ "DBInstanceIdentifier", "meanReadIOPS", "stdReadIOPS", "maxReadIOPS", "countReadIOPS" ]
df_agg_r


# for each DBInstanceIdentifier, get the average and maximum Write IOPS
df_agg_w = dfiops_write.groupby("DBInstanceIdentifier").Maximum.agg(["mean", "std", "max", "count"]).reset_index()
df_agg_w.columns = [ "DBInstanceIdentifier", "meanWriteIOPS", "stdWriteIOPS", "maxWriteIOPS", "countWriteIOPS" ]

df_iops = pd.merge(df_agg_r, df_agg_w, left_on="DBInstanceIdentifier", right_on="DBInstanceIdentifier", how="left")

df_iops

,DBInstanceIdentifier,meanReadIOPS,stdReadIOPS,maxReadIOPS,countReadIOPS,meanWriteIOPS,stdWriteIOPS,maxWriteIOPS,countWriteIOPS
0,g1,147.053128,967.970227,7504.800480,225,176.079372,1300.498092,10509.794107,225
1,g2,37.743779,271.922639,3157.056892,179,22.979451,141.187961,1339.233038,179
2,i1,245.008783,1431.490985,11555.003750,268,256.978313,1463.253014,9816.680278,268


In [17]:
df_c1 = pd.merge(df2, df_agg, left_on="DBInstanceIdentifier", right_on="DBInstanceIdentifier", how="left")

df_c1

,DBInstanceIdentifier,DBInstanceClass,Engine,DBInstanceStatus,AllocatedStorage,SecondaryAvailabilityZone,pricePerUnit,deploymentOption,vcpu,memory,pricePerMonth,meanCPU,stdCPU,maxCPU,countCPU
0,a1,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.590856,13.841096,99.761667,222
1,a2maz,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.087267,9.460886,99.770000,175
2,a2maz-ap-southeast-1b,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,3.986857,1.487108,23.253333,175
3,g1,db.r6g.xlarge,mysql,available,500,nan,0.510,Single-AZ,4.0,32.0,372.30,3.600990,13.481081,99.616667,224
4,g2,db.r6g.xlarge,mysql,available,500,ap-southeast-1c,1.020,Multi-AZ,4.0,32.0,744.60,2.109888,2.617055,27.020000,178
5,i1,db.r5.xlarge,mysql,available,500,nan,0.570,Single-AZ,4.0,32.0,416.10,5.323446,18.077301,99.228333,267


In [18]:
df_combined = pd.merge(df_c1, df_iops, left_on="DBInstanceIdentifier", right_on="DBInstanceIdentifier", how="left")

df_combined.meanReadIOPS = df_combined.meanReadIOPS.fillna(0)
df_combined.stdReadIOPS = df_combined.stdReadIOPS.fillna(0)
df_combined.maxReadIOPS = df_combined.maxReadIOPS.fillna(0)
df_combined.countReadIOPS = df_combined.countReadIOPS.fillna(0)

df_combined.meanWriteIOPS = df_combined.meanWriteIOPS.fillna(0)
df_combined.stdWriteIOPS = df_combined.stdWriteIOPS.fillna(0)
df_combined.maxWriteIOPS = df_combined.maxWriteIOPS.fillna(0)
df_combined.countWriteIOPS = df_combined.countWriteIOPS.fillna(0)

df_combined

,DBInstanceIdentifier,DBInstanceClass,Engine,DBInstanceStatus,AllocatedStorage,SecondaryAvailabilityZone,pricePerUnit,deploymentOption,vcpu,memory,pricePerMonth,meanCPU,stdCPU,maxCPU,countCPU,meanReadIOPS,stdReadIOPS,maxReadIOPS,countReadIOPS,meanWriteIOPS,stdWriteIOPS,maxWriteIOPS,countWriteIOPS
0,a1,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.590856,13.841096,99.761667,222,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,a2maz,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.087267,9.460886,99.770000,175,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,a2maz-ap-southeast-1b,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,3.986857,1.487108,23.253333,175,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,g1,db.r6g.xlarge,mysql,available,500,nan,0.510,Single-AZ,4.0,32.0,372.30,3.600990,13.481081,99.616667,224,147.053128,967.970227,7504.800480,225.0,176.079372,1300.498092,10509.794107,225.0
4,g2,db.r6g.xlarge,mysql,available,500,ap-southeast-1c,1.020,Multi-AZ,4.0,32.0,744.60,2.109888,2.617055,27.020000,178,37.743779,271.922639,3157.056892,179.0,22.979451,141.187961,1339.233038,179.0
5,i1,db.r5.xlarge,mysql,available,500,nan,0.570,Single-AZ,4.0,32.0,416.10,5.323446,18.077301,99.228333,267,245.008783,1431.490985,11555.003750,268.0,256.978313,1463.253014,9816.680278,268.0


In [19]:
# 2 GB RAM = 1 ACU, and very roughly, 1 ACU = 0.25 vCPU
# we currently do not recommend < 2 ACU for various reasons.. (although 0.5 ACU is the stated minimum)
df_combined['acu_usage'] = df_combined['meanCPU'] * df_combined['vcpu'] * 4 / 50
df_combined['acu_usage'] = df_combined['acu_usage'].apply(np.floor) + 0.5

# IOPS is read+write; add 20% buffer
df_combined['IOPS'] = (df_combined["meanReadIOPS"] + df_combined["meanWriteIOPS"]) * 1.2

df_combined

,DBInstanceIdentifier,DBInstanceClass,Engine,DBInstanceStatus,AllocatedStorage,SecondaryAvailabilityZone,pricePerUnit,deploymentOption,vcpu,memory,pricePerMonth,meanCPU,stdCPU,maxCPU,countCPU,meanReadIOPS,stdReadIOPS,maxReadIOPS,countReadIOPS,meanWriteIOPS,stdWriteIOPS,maxWriteIOPS,countWriteIOPS,acu_usage,IOPS
0,a1,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.590856,13.841096,99.761667,222,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.5,0.000000
1,a2maz,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.087267,9.460886,99.770000,175,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.5,0.000000
2,a2maz-ap-southeast-1b,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,3.986857,1.487108,23.253333,175,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.5,0.000000
3,g1,db.r6g.xlarge,mysql,available,500,nan,0.510,Single-AZ,4.0,32.0,372.30,3.600990,13.481081,99.616667,224,147.053128,967.970227,7504.800480,225.0,176.079372,1300.498092,10509.794107,225.0,1.5,387.759000
4,g2,db.r6g.xlarge,mysql,available,500,ap-southeast-1c,1.020,Multi-AZ,4.0,32.0,744.60,2.109888,2.617055,27.020000,178,37.743779,271.922639,3157.056892,179.0,22.979451,141.187961,1339.233038,179.0,0.5,72.867876
5,i1,db.r5.xlarge,mysql,available,500,nan,0.570,Single-AZ,4.0,32.0,416.10,5.323446,18.077301,99.228333,267,245.008783,1431.490985,11555.003750,268.0,256.978313,1463.253014,9816.680278,268.0,1.5,602.384515


In [20]:
### FIXME: haven't figured out how to extract ACU pricing from the Pricing API
### hard-wiring for now, currently APG/AMS Serverless V2 ACU pricing is identical
### also.. no GovCloud
### https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Concepts.RegionsAndAvailabilityZones.html

df_pricing = pd.DataFrame([
    ('us-east-1', 0.12, 0.20), # N. Virginia
    ('us-east-2', 0.12, 0.20), # Ohio
    ('us-west-1', 0.16, 0.22), # N. California
    ('us-west-2', 0.12, 0.20),  # Oregon
    ('af-south-1', 0.16, 0.26),  # Cape Town
    ('ap-east-1', 0.22, 0.24), # HK
    ('ap-south-2', 0.18, 0.22), # Hyderabad
    ('ap-southeast-3', 0.22, 0.22), # Jakarta
    ('ap-southeast-4', 0.20, 0.22), # Melbourne
    ('ap-south-1', 0.18, 0.22), # Mumbai
    ('ap-northeast-3', 0.20, 0.24), # Osaka
    ('ap-northeast-2', 0.20, 0.24), # Seoul
    ('ap-southeast-1', 0.20, 0.22), # Singapore
    ('ap-southeast-2', 0.20, 0.22), # Sydney
    ('ap-northeast-1', 0.20, 0.24), # Tokyo
    ('ca-central-1', 0.14, 0.22), # Canada
    ('eu-central-1', 0.14, 0.22), # Frankfurt
    ('eu-west-1', 0.14, 0.22), # Ireland
    ('eu-west-2', 0.14, 0.20), # London
    ('eu-south-1', 0.14, 0.23), # Milan
    ('eu-west-3', 0.14, 0.22), # Paris
    ('eu-south-1', 0.14, 0.22), # Spain
    ('eu-north-1', 0.14, 0.21), # Stockholm
    ('eu-central-2', 0.14, 0.242), # Zurich
    ('me-south-1', 0.15, 0.242), # Bahrain
    ('me-central-1', -1, 0.242), # UAE
    ('sa-east-1', 0.25, 0.28) # Sao Paolo
], columns=['regionCode', 'pricePerAcu', 'perPerMIOPS'])

df_pricing

,regionCode,pricePerAcu,perPerMIOPS
0,us-east-1,0.12,0.200
1,us-east-2,0.12,0.200
2,us-west-1,0.16,0.220
3,us-west-2,0.12,0.200
4,af-south-1,0.16,0.260
5,ap-east-1,0.22,0.240
6,ap-south-2,0.18,0.220
7,ap-southeast-3,0.22,0.220
8,ap-southeast-4,0.20,0.220
9,ap-south-1,0.18,0.220


In [21]:
# prevent this from bombing out unceremoniously if the current region is not one where ServerlessV2 is available
# (via the above hard-wired pricing list)
try:
    ppa = df_pricing[ df_pricing['regionCode'] == region ].values[0][1]
    ppiops = df_pricing[ df_pricing['regionCode'] == region ].values[0][2]
except KeyError:
    # large value to bloat the cost
    ppa = 999999
    ppiops = 999999
    
#df_combined['acuPricePerMonth'] = df_combined['acu_usage'] * ppa * 730

df_combined['acuPricePerMonth'] = np.where(df_combined['deploymentOption'] == 'Single-AZ',
                                           df_combined['acu_usage'] * ppa * 730,
                                          df_combined['acu_usage'] * ppa * 730 * 2)

# 730 hours/month x 3600 seconds/hour = 2628000 seconds
df_combined['miopsPricePerMonth'] = (df_combined['IOPS'] * 2628000 / 1000000) * ppiops
 
df_combined['aurora_serverless_PricePerMonth'] = df_combined['acuPricePerMonth'] + df_combined['miopsPricePerMonth']
df_combined['potentialSavings'] = df_combined['pricePerMonth'] - df_combined['aurora_serverless_PricePerMonth']

pid = os.getpid()
outputfile = "aurora_serverless_tco_%06d.csv" % pid
df_combined.to_csv(outputfile, index=False)

print("Wrote output file %s" % outputfile)
df_combined

Wrote output file aurora_serverless_tco_020793.csv


,DBInstanceIdentifier,DBInstanceClass,Engine,DBInstanceStatus,AllocatedStorage,SecondaryAvailabilityZone,pricePerUnit,deploymentOption,vcpu,memory,pricePerMonth,meanCPU,stdCPU,maxCPU,countCPU,meanReadIOPS,stdReadIOPS,maxReadIOPS,countReadIOPS,meanWriteIOPS,stdWriteIOPS,maxWriteIOPS,countWriteIOPS,acu_usage,IOPS,acuPricePerMonth,miopsPricePerMonth,aurora_serverless_PricePerMonth,potentialSavings
0,a1,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.590856,13.841096,99.761667,222,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.5,0.000000,219.0,0.000000,219.000000,238.710000
1,a2maz,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,5.087267,9.460886,99.770000,175,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.5,0.000000,219.0,0.000000,219.000000,238.710000
2,a2maz-ap-southeast-1b,db.r6g.xlarge,aurora-mysql,available,1,nan,0.627,Single-AZ,4.0,32.0,457.71,3.986857,1.487108,23.253333,175,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.5,0.000000,219.0,0.000000,219.000000,238.710000
3,g1,db.r6g.xlarge,mysql,available,500,nan,0.510,Single-AZ,4.0,32.0,372.30,3.600990,13.481081,99.616667,224,147.053128,967.970227,7504.800480,225.0,176.079372,1300.498092,10509.794107,225.0,1.5,387.759000,219.0,224.186743,443.186743,-70.886743
4,g2,db.r6g.xlarge,mysql,available,500,ap-southeast-1c,1.020,Multi-AZ,4.0,32.0,744.60,2.109888,2.617055,27.020000,178,37.743779,271.922639,3157.056892,179.0,22.979451,141.187961,1339.233038,179.0,0.5,72.867876,146.0,42.129291,188.129291,556.470709
5,i1,db.r5.xlarge,mysql,available,500,nan,0.570,Single-AZ,4.0,32.0,416.10,5.323446,18.077301,99.228333,267,245.008783,1431.490985,11555.003750,268.0,256.978313,1463.253014,9816.680278,268.0,1.5,602.384515,219.0,348.274631,567.274631,-151.174631
